In [ ]:
import commpy
import numpy as np
import scipy.interpolate 
#import tensorflow as tf
import math
import matplotlib.pyplot as plt
import os

#### Steps to simulate a coounication system

1. Bit Stream
1. Symbol Mapper
1. Transmitter
1. Channel
1. Estimator
1. Decoder

#### Things to be decided and fixed across all the simulations :

1. Bandwidth of the signal to be used: From Andrea Goldsmith example of 20MHz is used for all the subcarriers combines. We can use the same.

1. Frequency range of operation : (?)

In [ ]:
K = 64  # Number of carriers
CP = 16 # Cyclic Prefix
P = 4 # Carriers used for Pilot Symbols used
# Total of 48 sub carriers are available for data symbols

In [ ]:
def gen_bits(size):
    return np.random.randint(2,size=(size,))

In [ ]:
def OFDM_IFFT(symbol):
    return np.fft.ifft(symbol)
def OFDM_FFT(symbol):
    return np.fft.fft(symbol)
def addCP(x, len_cp):
    cp= x[-len_cp:]
    return np.append(cp,x)

In [ ]:
data_bits=gen_bits(48*2)  # Genrating the data bits to map it onto 48 subcarrier using QPSK
zero_bits = np.zeros((12*2,), dtype=int)  # Zero bits for inter-channel interference, something A.Goldsmith mentions
pilot_bits = np.zeros((4*2,), dtype = int) # Generating Pilot based bits
bit_frame = np.concatenate((pilot_bits,zero_bits,data_bits)) # One complete bit frame 

In [ ]:
QAM = commpy.QAMModem(4) # Creating QAM object for 4 symbol map i.e. 2 bits mapped onto each symbol
X=QAM.modulate(bit_frame) # Creating the Symbols from the bits, len = 64

In [ ]:
## Generating the OFDM frame with CP ###
x = OFDM_IFFT(X)
OFDM_frame= addCP(x,len_cp=16)

In [ ]:
### generating the channel ##
def rayleigh_channel(msg,ebno):
    ebno_lin = np.power(10,((ebno)/10)) 
    sigma = np.sqrt(1/(2*ebno_lin))
    h = (np.random.normal(0.0, 1.0, msg.shape) + 1j*np.random.normal(0.0, 1.0, msg.shape))/np.sqrt(2)
    n = (np.random.normal(0.0, sigma, msg.shape) + 1j*np.random.normal(0.0, sigma, msg.shape))/np.sqrt(2)

    return (np.multiply(msg,h) + n,h)

In [ ]:
channel_out,h = rayleigh_channel(OFDM_frame,10) # Energy per bit  

In [ ]:
Y_removeCP=channel_out[CP:]
y=OFDM_FFT(Y_removeCP)

### Cooperative OFDM 

1. Source
1. Relay
1. Destination

##### Protocol used for sending information

1. MultiHop (MxF) : First time slot S->R, second time slot R->D
1. Split Combine : in the first timeslot, the source transmits and only the relay is listening (just as in MxF). In the second timeslot, both source and relay transmit, and the destination is listening.
1. Diversity : in the first timeslot, the source transmits and both relay and destination are listening. In the second timeslot, only the relay is transmitting and the destination is listening. Thus, the destination gets two copies of the original signal.

##### Different Relay Methods we can use

1. Ampilify and Forward : The signal from the source node is received which is amplified by the following factor and then forwarded to the destination.

$$ A = \sqrt\frac{P_{R}}{P_{S}*h_{SR}^2 + P_{N}}  $$

The amplification taken is the limiting maximum value. 